<a href="https://colab.research.google.com/github/Heisnotanimposter/test_repo/blob/main/newPythonMOJOtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import h2o
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

h2o.init()

# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/bigdata/laptop/mnist/train.csv.gz")
test = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/bigdata/laptop/mnist/test.csv.gz")

# Identify predictors and response
x = train.columns[:-1]
y = "C785"

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Number of CV folds (to generate level-one data for stacking)
nfolds = 5

# Deep Learning with Cross-Validation and Stacked Ensembles
model = H2ODeepLearningEstimator(
    distribution="multinomial",
    activation="RectifierWithDropout",
    hidden=[32,32,32],
    input_dropout_ratio=0.2,
    sparse=True,
    l1=1e-5,
    epochs=10,
    nfolds=nfolds,
    fold_assignment="Modulo",
    keep_cross_validation_predictions=True,
    seed=1)

model.train(x=x, y=y, training_frame=train)

# Eval performance on the test set
perf = model.model_performance(test)

print(perf)